# Урок 3. Классификация. Логистическая регрессия.

## Домашнее задание <a class='anchor' id='hw'>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [4]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [5]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [6]:
def calc_logloss(y, y_pred):
    y_pred = y_pred.astype(np.float64)
#     print(y_pred)
    y_pred[y_pred==1] = 1 - 1e-5
    y_pred[y_pred<=1e-10] = 1e-5
#     print(y_pred)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.052685257853913287

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.


Изменим функцию eval_model, чтобы промежуточные результаты добавлялись о внешний список

In [7]:
result_list = []

In [8]:
np.random.seed(1)
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
#         if i % (iterations / 10) == 0:
#             print(i, W, err)
    result_list.append([i, eta, iterations, W, err])
    return W

Теперь можно пройтись вложенным циклом меняя гиперпараметры.

In [9]:
eta_list = np.geomspace(1e-5, 1000, num=30)
iter_list = np.arange(500, 500000, 5000)
# eta_list

In [10]:
# Ох и переборщил я тут с перебором :) сохраню только лучший результат

result_list = [[495499,
 6.2101694189156165,
 495500,
 [ 5.84269302,  2.392024  , -1.9952367 ,  6.95637343],
 1.1491483073229742e-06]]

# for iteration in iter_list:
#     for eta in eta_list:
#         W = eval_model(X_st, y, iterations=iteration, eta=eta)

In [11]:
min_eta = 1e10

for i, result in enumerate(result_list):
    if result[4] < min_eta:
        min_eta = result[4]
        min_eta_params = result
min_eta_params

[495499,
 6.2101694189156165,
 495500,
 [5.84269302, 2.392024, -1.9952367, 6.95637343],
 1.1491483073229742e-06]

Видим, что оптимальное eta находится в районе 6.2, что касается итерраций - для данной модели число получилось максимальное из заданных. Тестовых данных для проверки все равно нет, остановлюсь на этих параметрах и обучу модель.

In [12]:
W = eval_model(X_st, y, iterations=500000, eta=6.2)
W

array([-294.43093018,  -20.18131084,  -87.72903953,  224.18278007])

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).


Для определения вероятности найдем сигмоид для произведения весов на признаки, последние необходимо транспонировать.

In [13]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(W,X.T))
    return y_pred_proba


In [14]:
calc_pred_proba(W, X_st)


array([1.10851882e-02, 2.16174870e-18, 1.00000000e+00, 1.36040102e-45,
       9.99420193e-01, 1.74601372e-17, 1.00000000e+00, 1.07077081e-04,
       9.89596506e-01, 1.00000000e+00])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

То же самое что в 3 задании, но результат сравниваем с пороговым значением, и записываем 1, если сигмоид больше, 0 если меньше. Впрочем, вывод предыдущей функции показал, что от параметра t мало что зависит...

In [15]:
def calc_pred(W, X, t=0.5):
    y_pred = sigmoid(np.dot(W,X.T)) > t
    return y_pred.astype(int)

In [16]:
y_pred = calc_pred(W, X_st)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [17]:
y.astype(int)

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.


Модель получилась "суперточная", все меры равняются 1, поэтому я решил обучить ее на менее подходящих параметрах, чтобы появились ошибки, и найденные меры могли на них реагировать.

In [18]:
y_pred = calc_pred(eval_model(X_st, y, 500, 6.2), X_st)
y_pred

array([1, 0, 1, 0, 1, 0, 1, 1, 1, 1])

Accuracy это доля правильных ответов ко всем ответам.

In [19]:
true_y = sum(y_pred==y)
accuracy = true_y/len(y)
accuracy

0.8

Для построения матрицы ошибок, создадим список пар предсказанных / реальных значений

In [20]:
pair_y = list(zip(y_pred,y.astype(int)))

Создадим возможные комбинации и циклом будем инкрементировать одну из них, сравнивая пары

In [21]:
tp, tn, fp, fn = 0, 0, 0, 0

for pred, real in pair_y:
    if pred:
        if real:
            tp+= 1
        else:
            fn += 1
    else:
        if real:
            fp += 1
        else:
            tn +=1

print(f'tp: {tp}   tn: {tn} fp: {fp}   fn: {fn}')
            
            
            
            
    

tp: 5   tn: 3 fp: 0   fn: 2


Теперь по определениям:

In [22]:
precision = tp / (tp + fp)
recall = tp / (tp + fn)
print(f'precision: {precision}   recall: {recall}')

precision: 1.0   recall: 0.7142857142857143


Найдем F-меру по формуле:

In [23]:
f_score = 2 * precision * recall / (precision + recall)
f_score

0.8333333333333333

6. Могла ли модель переобучиться? Почему?

На датасете из 10 наблюдений сложно говорить о переобуении) данных слишком мало даже чтобы уловить закономерность, будут другие, тестовые данные - скорее всего модель начнет безбожно врать, но в данном случае это свидетельствует о малой обучающей выборке. 
Вообще, логистическая регрессия строит гиперплоскость, рассекая пространство признаков на два подпространства. После подбора оптимальных параметров, ситуация стабилизируется и больше плоскость двигаться почти не будет. 
Другое дело, если признаков много и не все из них значащие. Пытаясь найти закономерность случайном признаке, регрессия может приспособиться на трейне, и, ориентируясь на этот шум, начать подвирать на тесте. Такая ситуация была во второй части вебинара. Поэтому, прежде чем обучать модель, стоит посмотреть значимость признаков.